In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/queryresults/QueryResults.csv


Stack Overflow is a question-and-answer website for computer programmers.
It was created in 2008. It features questions and answers on certain computer programming topics.
As of March 2024 Stack Overflow has over 23 million registered users and has received over 24 million questions and 35 million answers.


For a dataset of 10,000 question-answer pairs, we propose to:

Perform document prepro
essing
Compare different LLM approaches (such as Gemini) to pred
ct tags
Implement, in the form of an API, a tag suggestion system for the site, ensuring that the questions are well-referenced and can be easily retrieved.

In [2]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=cff8cbd0a0ac3bf04891e3b8253e02783c1c180f1cef427433639817450be018
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [3]:
import pandas as pd
from transformers import pipeline
import swifter

In [4]:
df = pd.read_csv("/kaggle/input/queryresults/QueryResults.csv")
df.shape[0]

46500

In [5]:
df.head()

,Id,CreationDate,Score,ViewCount,AnswerCount,CommentCount,FavoriteCount,Title,Body,Tags
0,4,2008-07-31 21:42:52,795,72107,13,4,0.0,How to convert Decimal to Double in C#?,<p>I want to assign the decimal variable &quot...,<c#><floating-point><type-conversion><double><...
1,6,2008-07-31 22:08:08,319,23741,7,0,0.0,Why did the width collapse in the percentage w...,<p>I have an absolutely positioned <code>div</...,<html><css><internet-explorer-7>
2,9,2008-07-31 23:40:59,2211,797370,74,10,0.0,How do I calculate someone's age based on a Da...,<p>Given a <code>DateTime</code> representing ...,<c#><.net><datetime>
3,11,2008-07-31 23:55:37,1645,198982,41,3,0.0,Calculate relative time in C#,<p>Given a specific <code>DateTime</code> valu...,<c#><datetime><time><datediff><relative-time-s...
4,13,2008-08-01 00:42:38,704,285740,27,10,0.0,Determine a user's timezone,<p>Is there a standard way for a web server to...,<html><browser><timezone><user-agent><timezone...


## Nettoyage des tags

In [6]:
df['Tags'] = df['Tags'].apply(lambda x : x.replace('<', ''))
df['Tags'] = df['Tags'].apply(lambda x : x.replace('>', ','))
df['Tags'] = df['Tags'].apply(lambda x : x.split(','))
df['Tags']

0        [c#, floating-point, type-conversion, double, ...
1                       [html, css, internet-explorer-7, ]
2                                   [c#, .net, datetime, ]
3        [c#, datetime, time, datediff, relative-time-s...
4        [html, browser, timezone, user-agent, timezone...
                               ...                        
46495                                 [flash, swfobject, ]
46496         [python, django, dynamic, methods, lambda, ]
46497              [http, caching, etag, expires-header, ]
46498              [perl, algorithm, string, similarity, ]
46499    [algorithm, data-structures, binary-search-tre...
Name: Tags, Length: 46500, dtype: object

# Nettoyage des titres

# Nettoyage du body